In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt

list = os.listdir("Stocks1/")
number_files = len(list)

def archivo_no_vacio(fpath):
    return True if os.path.isfile(fpath) and os.path.getsize(fpath) > 0 else False

from collections import namedtuple

In [2]:
## hacemos un dict con los datos  de cada empresa

Empresa = namedtuple('Empresa', 'nombre industria sector marketcap')

empresas_df =  pd.read_csv("companies.csv", sep=",")
empresas_df["ticker"] = empresas_df["ticker"].str.lower()
# empresas_df

empresas_df = empresas_df[['ticker', 'company name', 'industry','sector', 'market cap']]
lista_nombres = empresas_df['company name'].tolist()
lista_tickers = empresas_df['ticker'].tolist()
lista_marketcap = empresas_df['market cap'].tolist()
lista_industrias = empresas_df['industry'].tolist()
lista_sectores = empresas_df['sector'].tolist()

dict_datos_empresas = dict()
for indice, ticker in enumerate(lista_tickers):

    nombre = lista_nombres[indice]
    industria = lista_industrias[indice]
    marketcap = lista_marketcap[indice]
    sector = lista_sectores[indice]

    dict_datos_empresas[ticker] = Empresa(nombre, industria, sector, marketcap)

# empresas_df


import json

with open("datos_empresas.json", "w") as file:
    dicto = json.dumps(dict_datos_empresas)
    file.write(dicto)

In [3]:
# con filtro fecha y filtro 500 empresas
list_files = os.listdir("Stocks1/")
dict_frames = dict() # dicccionario con todos los data frames que rellenaremos
dict_rendimientos = dict()

# no_sirven = 0
#recorremos todas las acciones en Stocks desde 2010 (Stocks1)
for nombre_archivo in list_files:

    # aplicamos filtro de 500 empresas
#     index = nombre_archivo.find(".us")
#     simple = nombre_archivo[0:index]
#     if simple not in list(empresas_df["Symbol"]):
#         continue

    path = f"Stocks1/{nombre_archivo}"

    # recorremos los archivos no vacios
    if archivo_no_vacio(path):

        df =  pd.read_csv(path, sep=",")
        df = df[['Date', 'Open', 'Close', 'Volume']]
        df['ticker'] = nombre_archivo.replace('.csv', '')
        df['Rendimiento'] = (df['Close'] - df['Open']) / df['Open']

        # quitamos los datos nulos
        df.dropna(subset=['Rendimiento'], inplace=True)

        df['Date']= pd.to_datetime(df['Date'], dayfirst = True)

        # filtramos para obtener los datos de hasta el 31 Dic de 2014
        df = df[(df['Date'] > '2015-01-01 00:00:00')]
        df = df[(df['Date'] < '2017-01-02 00:00:00')]

        # se guarda el nombre sin .txt
        dict_frames[nombre_archivo.split(".csv")[0]] = df
        dict_rendimientos[nombre_archivo.split(".csv")[0]] = df[['Rendimiento']]


In [4]:
# # encontremos la maxima presencia bursatil
# max_presencias = 0
# # hacemos un dict, key = nombre empresa, value = % presencia bursatil
# dict_presencias = dict()

# # calculo de la maxima presencia bursatil, valor que sera nuestra referencia
# #vemos las presencia de todas las acciones y elegimos la mayor
# for nombre, df in dict_frames.items():
#     # calculamos presencia bursatil de esa accion
#     pb = len(df.index)
#     if pb > max_presencias:
#         max_presencias = pb

# # guardamos el porcentaje de capitalizacion de cada empresa en el dict
# for nombre, df in dict_frames.items():
#     # calculamos presencia bursatil de esa accion
#     pb_porcentual = len(df.index)/max_presencias
#     dict_presencias[nombre] = pb_porcentual
with open('presencias.json') as json_file:
    dict_presencias = json.load(json_file)
print(len(dict_presencias))

7195


In [5]:
# filtramos con aquellas empresas que cumplen con el 99,9% de las presencias
# creamos un nuevo dict para evitar cambios del dict en la iteracion
dict_frames_nuevo = dict_frames.copy()
no_sirven = 0
c=0
p=0
for accion, df in dict_frames.items():
    # si tiene menos del 99,9% de presencias se elimina
    try:
        p+=1
        if dict_presencias[accion] < 0.999:
            del dict_frames_nuevo[accion]
    except:
        c+=1
        pass


# actualizamos las acciones
dict_frames = dict_frames_nuevo

#imprimimos la cantidad de acciones


# agregamos los datos de las empresas
dict_frames_nuevo = dict_frames.copy()
no_sirven = 0
for accion, df in dict_frames.items():
    if accion.split('.us')[0] not in dict_datos_empresas.keys():
        dict_frames_nuevo.pop(accion)
        no_sirven += 1
        continue
    dict_frames_nuevo[accion]['Nombre'] = dict_datos_empresas[accion.split('.us')[0]].nombre
    dict_frames_nuevo[accion]['Sector'] = dict_datos_empresas[accion.split('.us')[0]].sector
#     df['Market Cap'] = dict_datos_empresas[accion.split('.us')[0]].marketcap
# dict_frames_nuevo['a.us']

# df_prueba = dict_frames_nuevo['a.us'].iloc[0]
# df_prueba2 = dict_frames_nuevo['a.us'].iloc[-1]
# df_prueba
# df_prueba2
industrias_utilizadas = []
dic_rango_open= {}
for item in dict_frames_nuevo:
    if item not in industrias_utilizadas:
        industrias_utilizadas.append(item)
        df_open_inicio = dict_frames_nuevo[item].iloc[0]
        df_open_final = dict_frames_nuevo[item].iloc[-1]
        dic_rango_open[item] = [df_open_inicio['Open'],df_open_final['Open'], df_open_final['Open']-df_open_inicio['Open'] ]
    else:
        continue
dict_frames_nuevo
suma_ganancias = 0
total_acciones = 0
for item in dic_rango_open:
    suma_ganancias = +float(dic_rango_open[item][2])
    total_acciones+=1
promedio = suma_ganancias/total_acciones
print(suma_ganancias)
# dict_frames


-16.950000000000003


In [6]:
import json
with open("Resultados_acciones6.json") as json_file:
    resultados_acciones = json.load(json_file)
resultados_acciones

{'apu.us': 0.009055624983162307,
 'artna.us': 0.010901431818150807,
 'awk.us': 0.011942847704988975,
 'bip.us': 0.007784359021041255,
 'cpl.us': 0.004999999999999989,
 'duk.us': 0.01861405403643265,
 'ed.us': 0.02092089805642076,
 'kep.us': 0.004999999999999989,
 'nee.us': 0.010751758035044759,
 'ngg.us': 0.053681927409332024,
 'nwn.us': 0.004999999999999989,
 'pam.us': 0.004999999999999989,
 'ppl.us': 0.0132924952109907,
 'so.us': 0.04046152022630005,
 'sph.us': 0.01607942114923958,
 'sre.us': 0.011090191178237192,
 'utl.us': 0.005423471170658418,
 'bti.us': 0.01565953394305972,
 'cag.us': 0.005000000000000008,
 'casy.us': 0.005023901824114898,
 'ccu.us': 0.005000000000000008,
 'chd.us': 0.005000000000000008,
 'cl.us': 0.007227339804931753,
 'clx.us': 0.018497775144139382,
 'coke.us': 0.005000000000000008,
 'cpb.us': 0.005000000000000004,
 'dg.us': 0.005000000000000004,
 'flo.us': 0.005073397083293129,
 'hsy.us': 0.007899764214751932,
 'iba.us': 0.009303436196521339,
 'k.us': 0.009246

In [7]:
sum_ganancias2 = 0
total_acciones2 = 0
for item in resultados_acciones:
    
    if item != "parametros":
        sum_ganancias2 = +float(dic_rango_open[item][2]*resultados_acciones[item])
        total_acciones2+=1

promedio = sum_ganancias2/total_acciones2

print(resultados_acciones["parametros"])
print(promedio)
print(sum_ganancias2)

[0.02, 0.3, 0.0001779465, 0.05, 0.25, 0.0001779465]
0.0020776979151945524
0.17244892696114786
